In [1]:
%matplotlib inline

import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import datetime


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import sklearn.metrics
from sklearn.metrics import roc_auc_score


import warnings
warnings.filterwarnings('ignore')

### Загрузим тренировочные и тестовые данные

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train.head()

,Deal_id,Deal_date,First_deal_date,Secret_dwarf_info_1,Secret_dwarf_info_2,Secret_dwarf_info_3,First_default_date,Successful_deals_count,Region,Tavern,Hashed_deal_detail_1,Hashed_deal_detail_2,Hashed_deal_detail_3,Hashed_deal_detail_4,Hashed_deal_detail_5,Hashed_deal_detail_6,Age,Gender,Default
0,22487461,2015-11-05,2015-08-29,NaN,NaN,NaN,NaN,0.0,Tavern_district_3,7,2.5,-3,8,2.5,-3,5,36.0,Male,0
1,62494261,2016-08-26,2015-12-21,3.5,-2.0,5.0,2016-07-30,2.0,Tavern_district_4,7,2.5,-3,14,3.5,-3,5,29.0,Female,1
2,34822849,2016-02-18,2015-11-11,NaN,NaN,NaN,NaN,0.0,Tavern_district_6,7,2.5,-3,8,2.5,-3,5,56.0,Female,0
3,46893387,2016-04-30,2016-03-22,NaN,NaN,NaN,NaN,0.0,Tavern_district_2,13,2.5,-2,5,2.5,-3,5,27.0,Female,0
4,67128275,2016-09-19,2016-07-21,NaN,NaN,NaN,NaN,0.0,Tavern_district_4,39,2.5,-3,7,2.5,-3,5,37.0,Female,0


### Подготовка данных

In [2]:
### Напишем функцию, заполняющую пропуски в данных

def fill_missing_data(data):
    
### Захэшированную информацию по гномам - средним
    for feature in [1, 2, 3]:
        col_name = f"Secret_dwarf_info_{feature}"
        mean_ = data[col_name].mean()
        data[col_name] = data[col_name].fillna(mean_)
        
### Регион - самым популярным  
    mode_region_value = data.Region.mode()[0]
    data['Region'] = data['Region'].fillna(mode_region_value)
    
### Первую дату дефолта можно заполнить, например, каким-то
### "выбросом" (чтобы в признаковом пространстве подальше отделить
### объекты с пропуском в этой колонке)
    min_date_str = (
        data
        .First_default_date
        .dropna()
        .min()
    )
    
    min_date = datetime.datetime.strptime(
        min_date_str,
        '%Y-%m-%d'
    )
    
    date_for_missing_values = datetime.datetime(2015, 5, 1, 0, 0) - \
                              datetime.timedelta(days=365)
    
    date_ = str(date_for_missing_values)[:10]
    
    data['First_default_date'] = (
        data['First_default_date']
        .fillna(date_)
    )
    
### Количество успешных сделок - нулем
    data['Successful_deals_count'] = (
        data['Successful_deals_count']
        .fillna(0)
    )
    
    return 

In [3]:
### Применим функцию на тренировочных и тестовых данных

fill_missing_data(df_train)
fill_missing_data(df_test)

### Убедимся, что пропусков в данных нет:

df_train.isna().sum().sum(), df_test.isna().sum().sum()

(0, 0)

In [4]:
### Напишем функцию, генерирующую новые признаки 
### на основании базовых

def create_new_features(data):
    
    ### Распарсим даты в год-месяц-день   
    data["First_deal_date"] = pd.to_datetime(data["First_deal_date"])
    data['First_deal_year']= data['First_deal_date'].apply(lambda x: x.year)
    data['First_deal_month'] = data['First_deal_date'].apply(lambda x: x.month)
    data['First_deal_day'] = data['First_deal_date'].apply(lambda x: x.day)

    data["Deal_date"] = pd.to_datetime(data["Deal_date"])
    data['Deal_year']= data['Deal_date'].apply(lambda x: x.year)
    data['Deal_month'] = data['Deal_date'].apply(lambda x: x.month)
    data['Deal_day'] = data['Deal_date'].apply(lambda x: x.day)

    data["First_default_date"] = pd.to_datetime(data["First_default_date"])
    data['First_default_year']= data['First_default_date'].apply(lambda x: x.year)
    data['First_default_month'] = data['First_default_date'].apply(lambda x: x.month)
    data['First_default_day'] = data['First_default_date'].apply(lambda x: x.day)
    
    ### Создадим фичу
    ### "время от первой сделки до первой просрочки"
    ### в днях
    
    data['Difference'] = (data['First_default_date'] - data['First_deal_date']).dt.days
    
    ### Дропнем старые колонки с датами
    data.drop(
        [
            'First_deal_date',
            'Deal_date',
            'First_default_date'
        ],
        axis=1,
        inplace=True
    )
    
    return

In [5]:
### Применим функцию, убедимся в том, что
### Все трансформировалось ровно так, 
### как мы и планировали

create_new_features(df_train)
create_new_features(df_test)

df_train.head()

,Deal_id,Secret_dwarf_info_1,Secret_dwarf_info_2,Secret_dwarf_info_3,Successful_deals_count,Region,Tavern,Hashed_deal_detail_1,Hashed_deal_detail_2,Hashed_deal_detail_3,Hashed_deal_detail_4,Hashed_deal_detail_5,Hashed_deal_detail_6,Age,Gender,Default,First_deal_year,First_deal_month,First_deal_day,Deal_year,Deal_month,Deal_day,First_default_year,First_default_month,First_default_day,Difference
0,22487461,3.935514,-2.299065,5.26729,0.0,Tavern_district_3,7,2.5,-3,8,2.5,-3,5,36.0,Male,0,2015,8,29,2015,11,5,2014,5,1,-485
1,62494261,3.500000,-2.000000,5.00000,2.0,Tavern_district_4,7,2.5,-3,14,3.5,-3,5,29.0,Female,1,2015,12,21,2016,8,26,2016,7,30,222
2,34822849,3.935514,-2.299065,5.26729,0.0,Tavern_district_6,7,2.5,-3,8,2.5,-3,5,56.0,Female,0,2015,11,11,2016,2,18,2014,5,1,-559
3,46893387,3.935514,-2.299065,5.26729,0.0,Tavern_district_2,13,2.5,-2,5,2.5,-3,5,27.0,Female,0,2016,3,22,2016,4,30,2014,5,1,-691
4,67128275,3.935514,-2.299065,5.26729,0.0,Tavern_district_4,39,2.5,-3,7,2.5,-3,5,37.0,Female,0,2016,7,21,2016,9,19,2014,5,1,-812


In [6]:
### Напишем функцию, трансформирующую object
### колонки в вещественный/дискретный (числовой) формат
### Трансформировать прочие числовые (но категориальные)
### фичи не будем, так как планируем строить композиции деревьев
### в качестве моделей


# Посчитаем средние по тренировочному датасету
# т.к. на тесте средние таргетов взять невозможно

mean_gender = df_train.groupby('Gender')['Default'].mean()

def transform_object_cols(data, means=mean_gender):
    
    data['Gender'] = data['Gender'].map(mean_gender)
    
    dummy = pd.get_dummies(data.Region, drop_first=True)    
    data.drop('Region', axis=1, inplace=True)
    
    data[dummy.columns] = dummy
    
    return

In [7]:
### Применим функцию, убедимся в том, что
### Все трансформировалось ровно так, 
### как мы и планировали

transform_object_cols(df_train)
transform_object_cols(df_test)

df_train.head()

,Deal_id,Secret_dwarf_info_1,Secret_dwarf_info_2,Secret_dwarf_info_3,Successful_deals_count,Tavern,Hashed_deal_detail_1,Hashed_deal_detail_2,Hashed_deal_detail_3,Hashed_deal_detail_4,Hashed_deal_detail_5,Hashed_deal_detail_6,Age,Gender,Default,First_deal_year,First_deal_month,First_deal_day,Deal_year,Deal_month,Deal_day,First_default_year,First_default_month,First_default_day,Difference,Tavern_district_1,Tavern_district_2,Tavern_district_3,Tavern_district_4,Tavern_district_5,Tavern_district_6,Tavern_district_7
0,22487461,3.935514,-2.299065,5.26729,0.0,7,2.5,-3,8,2.5,-3,5,36.0,0.168565,0,2015,8,29,2015,11,5,2014,5,1,-485,False,False,True,False,False,False,False
1,62494261,3.500000,-2.000000,5.00000,2.0,7,2.5,-3,14,3.5,-3,5,29.0,0.101502,1,2015,12,21,2016,8,26,2016,7,30,222,False,False,False,True,False,False,False
2,34822849,3.935514,-2.299065,5.26729,0.0,7,2.5,-3,8,2.5,-3,5,56.0,0.101502,0,2015,11,11,2016,2,18,2014,5,1,-559,False,False,False,False,False,True,False
3,46893387,3.935514,-2.299065,5.26729,0.0,13,2.5,-2,5,2.5,-3,5,27.0,0.101502,0,2016,3,22,2016,4,30,2014,5,1,-691,False,True,False,False,False,False,False
4,67128275,3.935514,-2.299065,5.26729,0.0,39,2.5,-3,7,2.5,-3,5,37.0,0.101502,0,2016,7,21,2016,9,19,2014,5,1,-812,False,False,False,True,False,False,False


### Обучение модель

In [8]:
### Разделим таргеты и фичи

X_train = df_train.drop(['Default', 'Deal_id'], axis=1)
X_test = df_test.drop(['Deal_id'], axis=1)

Y_train = df_train['Default']

In [9]:
from sklearn.ensemble import RandomForestClassifier

max_test_score = 0

model = RandomForestClassifier(
    random_state=472
)

model.fit(X_train, Y_train)

RandomForestClassifier(random_state=472)

Даже такой простейшей модели хватает, чтобы побить бейзлайн задачи.
RandomState - единственный параметр Случайного Леса, по которому производилась валидация.

Можно заметно улучшить результаты, скажем, добавив больше фичей и дополнительно провалидировавшись на остальных гиперпараметрах + моделях.

### Загрузка данных в csv нужного формата

In [10]:
submission = df_test['Deal_id'].copy().to_frame()
submission['Prediction'] = model.predict_proba(X_test)[:, 1]

submission.head()

,Deal_id,Prediction
0,72875713,0.10
1,75825544,0.09
2,81809181,0.24
3,87083256,0.19
4,84651519,0.29


In [11]:
submission.to_csv('submission.csv', index=False)